In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import joblib
import os

In [ ]:
# 1. Load Data
try:
    df_hypertension = pd.read_csv('hypertension_data.csv')
    print("Hypertension data loaded successfully.")
except FileNotFoundError:
    print("File hypertension_data.csv not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Hypertension data loaded successfully.


In [ ]:
#2.EDA
print(df_hypertension.head())
print(df_hypertension.info())
print(df_hypertension.isnull().sum())
print(df_hypertension['target'].value_counts())

    age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0  57.0  1.0   3       145   233    1        0      150      0      2.3   
1  64.0  0.0   2       130   250    0        1      187      0      3.5   
2  52.0  1.0   1       130   204    0        0      172      0      1.4   
3  56.0  0.0   1       120   236    0        1      178      0      0.8   
4  66.0  0.0   0       120   354    0        1      163      1      0.6   

   slope  ca  thal  target  
0      0   0     1       1  
1      0   0     2       1  
2      2   0     2       1  
3      2   0     2       1  
4      2   0     2       1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26083 entries, 0 to 26082
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       26083 non-null  float64
 1   sex       26058 non-null  float64
 2   cp        26083 non-null  int64  
 3   trestbps  26083 non-null  int64  
 4   chol      26083 non-null  i

In [ ]:
# 3. Preprocessing

X_hypertension = df_hypertension.drop('target', axis=1)
y_hypertension = df_hypertension['target']

categorical_cols_hypertension = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']
numerical_cols_hypertension = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

for col in numerical_cols_hypertension:
    if X_hypertension[col].isnull().any():
        X_hypertension[col] = X_hypertension[col].fillna(X_hypertension[col].median())
for col in categorical_cols_hypertension:
    if X_hypertension[col].isnull().any():
        X_hypertension[col] = X_hypertension[col].fillna(X_hypertension[col].mode()[0])

preprocessor_hypertension = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols_hypertension),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols_hypertension)
    ],
    remainder='passthrough' # Keep other columns (if any)
)

X_processed_hypertension = preprocessor_hypertension.fit_transform(X_hypertension)

X_train_hypertension, X_test_hypertension, y_train_hypertension, y_test_hypertension = train_test_split(
    X_processed_hypertension, y_hypertension, test_size=0.2, random_state=42, stratify=y_hypertension
)

print(f"Hypertension X_train shape: {X_train_hypertension.shape}")
print(f"Hypertension X_test shape: {X_test_hypertension.shape}")

Hypertension X_train shape: (20866, 30)
Hypertension X_test shape: (5217, 30)


In [ ]:
# 4.Membuat Model
model_hypertension = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_hypertension.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid') 
])

model_hypertension.compile(optimizer=Adam(learning_rate=0.001),
                          loss='binary_crossentropy',
                          metrics=['accuracy'])

print("\nHypertension Model Summary:")
model_hypertension.summary()

print("\nTraining Hypertension Model...")
history_hypertension = model_hypertension.fit(
    X_train_hypertension, y_train_hypertension,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=0
)

print(f"Hypertension Training Accuracy: {history_hypertension.history['accuracy'][-1]:.4f}")
print(f"Hypertension Validation Accuracy: {history_hypertension.history['val_accuracy'][-1]:.4f}")



Hypertension Model Summary:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               3968      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 14337 (56.00

In [ ]:
# Evaluasi
loss_hypertension, accuracy_hypertension = model_hypertension.evaluate(X_test_hypertension, y_test_hypertension, verbose=0)
print(f"\nHypertension Test Loss: {loss_hypertension:.4f}")
print(f"Hypertension Test Accuracy: {accuracy_hypertension:.4f}")


Hypertension Test Loss: 0.0000
Hypertension Test Accuracy: 1.0000


In [ ]:
# 8. menyimpan model dan preprocessor
model_dir_hypertension = './saved_models/hypertension_model'
os.makedirs(model_dir_hypertension, exist_ok=True)
model_hypertension.save(model_dir_hypertension)
print(f"Hypertension model saved to {model_dir_hypertension}")

preprocessor_hypertension_path = './saved_models/hypertension_preprocessor.pkl'
joblib.dump(preprocessor_hypertension, preprocessor_hypertension_path)
print(f"Hypertension preprocessor saved to {preprocessor_hypertension_path}")

INFO:tensorflow:Assets written to: ./saved_models/hypertension_model\assets


INFO:tensorflow:Assets written to: ./saved_models/hypertension_model\assets


Hypertension model saved to ./saved_models/hypertension_model
Hypertension preprocessor saved to ./saved_models/hypertension_preprocessor.pkl
